In [1]:
import pandas as pd
import psycopg2

In [2]:
from pyspark.sql import DataFrameReader
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import *

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
df_i94=spark.read.parquet("sas_data")
df_i94.head(5)

[Row(cicid=5748517.0, i94yr=2016.0, i94mon=4.0, i94cit=245.0, i94res=438.0, i94port='LOS', arrdate=20574.0, i94mode=1.0, i94addr='CA', depdate=20582.0, i94bir=40.0, i94visa=1.0, count=1.0, dtadfile='20160430', visapost='SYD', occup=None, entdepa='G', entdepd='O', entdepu=None, matflag='M', biryear=1976.0, dtaddto='10292016', gender='F', insnum=None, airline='QF', admnum=94953870030.0, fltno='00011', visatype='B1'),
 Row(cicid=5748518.0, i94yr=2016.0, i94mon=4.0, i94cit=245.0, i94res=438.0, i94port='LOS', arrdate=20574.0, i94mode=1.0, i94addr='NV', depdate=20591.0, i94bir=32.0, i94visa=1.0, count=1.0, dtadfile='20160430', visapost='SYD', occup=None, entdepa='G', entdepd='O', entdepu=None, matflag='M', biryear=1984.0, dtaddto='10292016', gender='F', insnum=None, airline='VA', admnum=94955622830.0, fltno='00007', visatype='B1'),
 Row(cicid=5748519.0, i94yr=2016.0, i94mon=4.0, i94cit=245.0, i94res=438.0, i94port='LOS', arrdate=20574.0, i94mode=1.0, i94addr='WA', depdate=20582.0, i94bir=29.

In [5]:
# After running create_tables.py, insert the data into the database
import configparser
config = configparser.ConfigParser()
config.read('dwh.cfg')

['dwh.cfg']

In [6]:
#urlstr ="jdbc:redshift://redshifthost:5439/database?user=username&password=pass"
urlstr ="jdbc:redshift://" + config['CLUSTER']['host'] + ":5439/" + config['CLUSTER']['dbname'] + "?user=" + config['CLUSTER']['user'] + "&password=" + config['CLUSTER']['password']
print(urlstr)
#s3addr="s3n://path/for/temp/data"
s3addr= config['Roles']['s3']
print(s3addr)

jdbc:redshift://dwhcluster.ck30nj6o0zy2.us-west-2.redshift.amazonaws.com:5439/dwh?user=dwhuser&password=Passw0rd
s3://udacity-dend-capstone-etl/temp/


In [7]:
df_i94.write \
  .format("com.databricks.spark.redshift") \
  .option("url", urlstr) \
  .option("dbtable", "my_table_copy") \
  .option("tempdir", s3addr) \
  .mode("error") \
  .save()

Py4JJavaError: An error occurred while calling o52.save.
: java.lang.ClassNotFoundException: Failed to find data source: com.databricks.spark.redshift. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:657)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:245)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at 

### WORLD Temperature 

In [ ]:
df_temp = pd.read_csv('./data/GlobalLandTemperaturesByCity.csv')
df_temp_us = df_temp[df_temp["Country"] == "United States"]
df_temp_us.head()